# **Data Skew and Data Movement**

 This notebook:

1. shows data skew and explain round\_robin on small tables
2. shows execution plans when querying round\_robin
3. shows execution plans when querying round\_robins joined

DMVs used in this notebook:

- [sys.schemas](https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/schemas-catalog-views-sys-schemas?view=azure-sqldw-latest?WT.mc_id=DP-MVP-5004236): Contains a row for each database schema.
    
- [sys.tables](https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-tables-transact-sql?view=azure-sqldw-latest?WT.mc_id=DP-MVP-5004236): Returns a row for each user table in SQL Server. 
    
- [sys.pdw\_table\_distribution\_propertie](https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-pdw-table-distribution-properties-transact-sql?view=aps-pdw-2016-au7?WT.mc_id=DP-MVP-5004236)s: Holds distribution information for tables.
    
- [sys.pdw\_table\_mappings](https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-pdw-table-mappings-transact-sql?view=aps-pdw-2016-au7?WT.mc_id=DP-MVP-5004236): Ties user tables to internal object names by object\_id.
    
- [sys.pdw\_nodes\_tables](https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-pdw-nodes-tables-transact-sql?view=aps-pdw-2016-au7?WT.mc_id=DP-MVP-5004236): Contains a row for each table object that a principal either owns or on which the principal has been granted some permission.
    
- [sys.dm\_pdw\_nodes\_db\_partition\_stats](https://learn.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-db-partition-stats-transact-sql?view=sql-server-ver16?WT.mc_id=DP-MVP-5004236): Returns page and row-count information for every partition in the current database.
    
- [sys.dm\_pdw\_exec\_requests](https://learn.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-pdw-exec-requests-transact-sql?view=aps-pdw-2016-au7?WT.mc_id=DP-MVP-5004236): Holds information about all requests currently or recently active in synapse.
    
- [sys.dm\_pdw\_request\_steps](https://learn.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-pdw-request-steps-transact-sql?view=aps-pdw-2016-au7?WT.mc_id=DP-MVP-5004236): Holds information about all steps that compose a given request or query
    
- [DBCC PDW\_SHOWSPACEUSED](https://learn.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-pdw-showspaceused-transact-sql?view=aps-pdw-2016-au7?WT.mc_id=DP-MVP-5004236): Displays the number of rows, disk space reserved, and disk space used for a specific table

# Query 1

## <span style="font-size: 14px;">lists tables and their distribution policy</span>

In [ ]:
-- Query 1
select
    schema_name = s.name 
    ,table_name = t.name 
    ,distribution_policy = tp.distribution_policy_desc
    ,row_count = sum(row_count) 
    ,max_row_count_per_distribution = max(row_count) 
    ,min_row_count_per_distribution = min(row_count) 
    ,avg_row_count_per_distribution = avg(row_count)
from sys.schemas s
    inner join sys.tables as t on s.schema_id = t.schema_id
    inner join sys.pdw_table_distribution_properties as tp on t.object_id = tp.object_id
    inner join sys.pdw_table_mappings as tm on t.object_id = tm.object_id
    inner join sys.pdw_nodes_tables as nt on tm.physical_name = nt.name
    inner join sys.dm_pdw_nodes_db_partition_stats as nps on nt.object_id = nps.object_id
        and nt.pdw_node_id = nps.pdw_node_id
        and nt.distribution_id = nps.distribution_id
group by 
    s.name
    ,t.name
    ,tp.distribution_policy_desc;

# Query 2

evaluates data skewness per table

In [ ]:
-- Query 2
with t as 
(
    select
        schema_name = s.name 
        ,table_name = t.name 
        ,distribution_policy = tp.distribution_policy_desc
        ,row_count = sum(row_count) 
        ,max_row_count_per_distribution = max(row_count) 
        ,min_row_count_per_distribution = min(row_count) 
        ,avg_row_count_per_distribution = avg(row_count)
    from sys.schemas s
        inner join sys.tables as t on s.schema_id = t.schema_id
        inner join sys.pdw_table_distribution_properties as tp on t.object_id = tp.object_id
        inner join sys.pdw_table_mappings as tm on t.object_id = tm.object_id
        inner join sys.pdw_nodes_tables as nt on tm.physical_name = nt.name
        inner join sys.dm_pdw_nodes_db_partition_stats as nps on nt.object_id = nps.object_id
            and nt.pdw_node_id = nps.pdw_node_id
            and nt.distribution_id = nps.distribution_id
    group by 
        s.name
        ,t.name
        ,tp.distribution_policy_desc
)
select 
    t.*
	,table_skew = case 
        when row_count = 0 then -1
	    else abs(max_row_count_per_distribution * 1.0 - min_row_count_per_distribution * 1.0) / max_row_count_per_distribution * 100.0
	end
from t;

# Query 3

Displays the number of rows, disk pasce reserved and disk space used for dbo.Trip and dbo.Date table.

This feature give insights on table distribution.

In [ ]:
-- Query 3
DBCC PDW_SHOWSPACEUSED('dbo.Trip')
DBCC PDW_SHOWSPACEUSED('dbo.Date')

# Query 4

Performs a query over a round robin table

In [ ]:
-- Query 4
select top(1000) * 
from dbo.trip 
option(label = 'SELECT FROM ROUND_ROBIN')

# Query 5

Get execution info for the query using its label

In [ ]:
-- Query 5
select * 
from sys.dm_pdw_exec_requests  as t 
where t.[label] = 'SELECT FROM ROUND_ROBIN' 
order by start_time desc 

# Query 6

Gets execution plan for the query using its request\_id

In [ ]:
-- Query 6
declare @request_id as nvarchar(50) = 'QID21315'

select * 
from sys.dm_pdw_request_steps as t 
where request_id = @request_id;

# Query 7

Query 5 and 6 in one step for dbo.Geography

In [ ]:
-- Query 7
select * 
from dbo.[Geography] 
option(label = 'SELECT FROM ROUND_ROBIN')

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'SELECT FROM ROUND_ROBIN'
    order by start_time desc 
)

# Query 8

Shows plan of Round Robin joined to another Round Robin

In [ ]:
-- Query 8
select top(100000) * 
from dbo.Trip as t 
    inner join dbo.Geography as g on t.PickupGeographyID = g.GeographyID
option (label = 'SELECT FROM ROUND_ROBIN JOIN ROUND_ROBIN');

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'SELECT FROM ROUND_ROBIN JOIN ROUND_ROBIN'
    order by start_time desc 
)

# Query 9

Shows plan of a Round Robin joined to 2 Round Robins

In [ ]:
-- Query 9
select top(100000) * 
from dbo.Trip as t 
    inner join dbo.Geography as g on t.PickupGeographyID = g.GeographyID
    inner join dbo.HackneyLicense as h on t.HackneyLicenseID = h.HackneyLicenseID
option (label = 'SELECT FROM ROUND_ROBIN JOIN ROUND_ROBINx2');

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'SELECT FROM ROUND_ROBIN JOIN ROUND_ROBINx2'
    order by start_time desc 
)